#### 아래 내용은 Intel® AI for Youth 프로그램을 참고하여
BrainAI와 BrainAI Coach Network에서 개발한 내용입니다.<br> 
상업적 사용은 불가하며 교육기관 및 학교에서 학생들 교육활동에 자유롭게 사용가능합니다.

# 프로젝트 제목: 새로운 영상 데이터(M&M 초콜릿)를 불러와 관심 영역을 설정하여 예측 테스트
새로운 영상 데이터(M&M 초콜릿)를 불러와 관심 영역을 설정하여 Teachable Machine에서 개발한 모델을 활용하여 예측하는 파이썬 프로그램 작성하기

### Task 1. 필요 라이브러리 불러오기

In [4]:
import cv2
import numpy as np
import tensorflow.keras
from tensorflow.keras.models import model_from_json
print("프로젝트 수행에 필요한 라이브러리를 모두 불러왔습니다.")

프로젝트 수행에 필요한 라이브러리를 모두 불러왔습니다.


### Task 2. Teachable Machine에서 개발한 모델(model)과 클래스(label) 불러오기 함수 작성

In [5]:
def loadModel(): 
    model = tensorflow.keras.models.load_model('model/keras_model.h5', compile = False) 
    labels = []
    file = open("model/labels.txt", "r")
    for x in file:
        labels.append(x.rstrip('\n'))
    print('labels = ', labels)
    file.close()
    return model, labels

### Task 3. 'Task 2.'함수 실행하여 결과 확인 하기

In [6]:
model, labels = loadModel()

labels =  ['0 None', '1 Red', '2 Orange', '3 Yellow', '4 Green', '5 Blue', '6 Brown']


### Task 4. 새로운 이미지 데이터(M&M 초콜릿) Color 예측하기 함수 작성

In [7]:
def ReadPicture(cropped, model):   
    
    input_width = 224
    input_height = 224

    resized_image = cv2.resize(cropped, (input_width, input_height))
    imgRGB =cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
    
    normalized_image = (imgRGB.astype(np.float32) / 127.0) - 1
    batch = normalized_image.reshape(1, input_height, input_width, 3)

    prediction = model.predict(batch) 

    class_id = np.argmax(prediction)
    score = np.max(prediction) 
    
    return class_id, score

### Task 5. 새로운 이미지 데이터(M&M 초콜릿) 불러와 Color 예측하기

In [8]:
# 새로운 이미지 데이터 불러오기
cropped = cv2.imread("images/green.jpg")

In [9]:
# 예측 실행하여 결과 저장하기
class_id, score = ReadPicture(cropped, model)   
print("Class:", class_id)
print("Confidence:", score)

# 결과 출력하기
text = 'AI 예측 결과는  ' + labels[class_id][2:] + ' ' + str(int(score*100)) + '%' + '입니다' 
print(text)

# 예측에 사용된 이미지 데이터 확인 하기
cv2.imshow("Input Data", cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()

Class: 4
Confidence: 0.9999982
AI 예측 결과는  Green 99%입니다


### Task 6. 새로운 영상 데이터(M&M 초콜릿) 불러와 Color 예측하기

In [10]:
import time

def Main():
    video_capture = cv2.VideoCapture(video)
    tic = time.time()
    while(True):
        grabbed, frame = video_capture.read()
        frame = cv2.flip(frame, 1)
        
        if time.time() - tic > timeDelay: # 3초마다 예측 실행
            # 예측 실행하여 결과 저장하기
            class_id, score = ReadPicture(frame, model)   
            print("Class:", class_id)
            print("Confidence:", score)

            # 결과 출력하기
            text = 'AI 예측 결과는  ' + labels[class_id][2:] + ' ' + str(int(score*100)) + '%' + '입니다'
            print(text)
            tic = time.time()

        cv2.imshow("Original Flip Video", frame)
        key = cv2.waitKey(1)
        if key == ord("q"):
            break
            
    video_capture.release()
    cv2.destroyAllWindows()

In [11]:
# 설정 값 변경 및 프로그램 실행
video = 0 #노트북 built in 웹캠, 외부 연결 웹캠인 경우 1로 수정
timeDelay = 3

if __name__ == '__main__': 
    Main()

Class: 0
Confidence: 0.5349415
AI 예측 결과는  None 53%입니다


### Task 7. 새로운 영상 데이터(M&M 초콜릿) 관심 영역 설정하여 예측하기

In [18]:
import time

def Main():
    video_capture = cv2.VideoCapture(video)
    tic = time.time()
    
    # ---- 영상안에 p 눌러 시작하는 텍스트 출력 ---- #    
    color  = (0, 0, 255)
    text = "Press [p] to start"
    print(text) 
        
    # ---- 불러온 영상 크기 값 저장 ---- #    
    height = video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
    width = video_capture.get(cv2.CAP_PROP_FRAME_WIDTH)
    
    # ---- 영상에서 매개 변수 top, bottom 값을 이용하여 관심 영역 설정 ---- #   
    y1 = int(height - (height * top))
    y2 = int(height - (height * bottom))
    square = (y2-y1)/2
    x1 = int(width/2 - square)
    x2 = int(width/2 + square)
    
    # ---- MODE를 이용하여 실행, 멈춤을 컨트롤 ---- #
    MODE_PAUSE = -1 
    MODE_START = 0
    mode_status = MODE_PAUSE
    
    while(True):
        grabbed, frame = video_capture.read()
        frame = cv2.flip(frame, 1)
        
        # ---- 관심 영역을 변수 cropped에 저장  ---- #
        cropped = frame[y1:y2, x1:x2]
        # ---- 관심 영역에 박스 그리기  ---- #
        cv2.rectangle(frame, (x1,y1), (x2,y2), color, 2)
        # ---- 관심 영역에 텍스트(text변수에 저장된 값) 출력 하기  ---- #
        cv2.putText(frame, text, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, 2)  
        # ---- 모델을 활용한 예측 실행  ---- #
        if mode_status == MODE_START and time.time() - tic > timeDelay: # 3초마다 예측 실행
            cv2.imshow("cropped", cropped)
            # 예측 실행하여 결과 저장하기
            class_id, score = ReadPicture(cropped, model)   
            print("Class:", class_id)
            print("Confidence:", score)

            # 결과 출력하기
            result_text = 'AI 예측 결과는  ' + labels[class_id][2:] + ' ' + str(int(score*100)) + '%' + '입니다'
            print(result_text)
            tic = time.time()

        cv2.imshow("Original Flip Video", frame)
        key = cv2.waitKey(1)
        if key == ord("q"):
            break
            
        # ---- 프로그램 실행 시작 또는 잠시 멈춤 설정(토글) ---- #
        elif key == ord('p'):
            if mode_status != MODE_PAUSE: 
                mode_status = MODE_PAUSE       
                color = (0, 0, 255)
                text = "Press 'p' to start"

            elif mode_status == MODE_PAUSE:
                mode_status = MODE_START
                color = (0, 255, 0)
                text = "Press 'p' to pause"
    
    video_capture.release()
    cv2.destroyAllWindows()

In [19]:
# 설정 값 변경 및 프로그램 실행
video = 0 #노트북 built in 웹캠, 외부 연결 웹캠인 경우 1로 수정
timeDelay = 3

#---- 관심 영역 설정 값 ----#
top = .65            
bottom = .25

#---- 프로그램 실행 ----#
if __name__ == '__main__': 
    Main()

Press [p] to start
